<a href="https://colab.research.google.com/github/jason96819/Project_1/blob/main/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%99%95%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글드라이브 마운트하기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 사전 학습 모델 load

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertModel
from transformers import BertTokenizerFast
import pandas as pd

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'

In [ ]:
# 토크나이저 지정
tokenizer_pretrained = CHECKPOINT_NAME

In [ ]:
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [ ]:
# CustomBertModel 생성
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)

CustomBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
# 저장한 state_dict를 로드 합니다.
bert.load_state_dict(torch.load("./drive/MyDrive/data/project/bert_model.pth"))

<All keys matched successfully>

In [ ]:
class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels

    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        prob = prediction.max(dim=1)[0].item()
        prob_1 = f'{prob*100:.3f}'
        output = prediction.argmax(dim=1).item()
        result = self.labels[output]
        return result, prob_1
        #print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

# Huggingface 토크나이저 생성
tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)

labels = {
    0: '0',             # 부정
    1: '1'              # 긍정
}

# CustomPredictor 인스턴스를 생성합니다.
predictor = CustomPredictor(bert, tokenizer, labels)

In [ ]:
def sentiment(x):
    res, pro = predictor.predict(x)
    return res, pro

# 데이터 확인 및 전처리

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('./drive/MyDrive/data/project/maple.csv')
df = df[['Date', 'Reaction']]
df

,Date,Reaction
0,2023-06-01 00:00,아 맞다 금은손 헉
1,2023-06-01 00:00,1살 or 2살 어려진 거 축하
2,2023-06-01 00:00,멍눈은 도대체 언제나오냐 최근에 나온적 있음?
3,2023-06-01 00:00,내일부터 진짜 공부함 일주일전부터 공부시작이지 원래
4,2023-06-01 00:00,왜 졸리지 반나절을 잣는데 우째서
...,...,...
177631,2023-06-30 23:59,메m) 170 달성하신 형님들 꿀팁 알려주십쇼ㅠ 29일 부터 사냥만 죽어라 해서 1...
177632,2023-06-30 23:59,님들 저 자랑할 거 있어여 버닝섭 몬파 상자에서 재획비 먹음!!펫하나 더 살 수 있다!
177633,2023-06-30 23:59,내가 오마이걸팬은 아닌데 오마이걸노래 좋더라 https://youtu.be/xnAY...
177634,2023-06-30 23:59,그냥 육성법 대중적으로 하는 그대로 따라갈까 에반 키우고 있었는데 퀘 깨다가 이거 ...


In [ ]:
# '6차'단어만 포함된 데이터 프레임 추출
target_df = df[df['Reaction'].str.contains('6차')]
target_df = target_df.reset_index(drop=True)
target_df

,Date,Reaction
0,2023-06-01 01:02,6차 아니면 로드맵 유기하면 안되는거 아니야 간담회까지 열어서 로드맵 보여주겠다고 ...
1,2023-06-01 02:08,6차 왔냐 66 66 6 6 ;;;
2,2023-06-01 02:59,근데 6차나오면 전직업 최종뎀 뻥튀기되서 보스컷 낮아지지않음? 5차때처럼 5차스킬하...
3,2023-06-01 03:01,6차 나올때까지 노말 진듄다니기 vs 하드 진듄 2인팟 가기 어카지? 한달만 참어??
4,2023-06-01 03:34,6차 거의 확정으로보는 분위기 아녓음? 오늘은분위기가 갑자기 다들 회의적이네 냉정해졌어
...,...,...
15070,2023-06-30 23:19,오늘 지하철 보니까 버츄얼 유튜버 생일도 챙기고 그러던데 우리도 6차 잘나오면 지하...
15071,2023-06-30 23:26,●▅▇█▇▆▅▄▇ 아 6차 배달 잘 오고 있나?
15072,2023-06-30 23:47,진짜 메이플 빼고 옆동네는 다 난리났네… 로아는 안그래도 망해가는데 중국몽 같이해버...
15073,2023-06-30 23:57,진짜 심각) 미하일 주력기 문의 한번만 더 봐주세요 재업 추하지만 저번에 올렸던 글...


In [ ]:
# 길이 추출
target_df['Length'] = target_df['Reaction'].str.len()
target_df

,Date,Reaction,Length
0,2023-06-01 01:02,6차 아니면 로드맵 유기하면 안되는거 아니야 간담회까지 열어서 로드맵 보여주겠다고 ...,176
1,2023-06-01 02:08,6차 왔냐 66 66 6 6 ;;;,19
2,2023-06-01 02:59,근데 6차나오면 전직업 최종뎀 뻥튀기되서 보스컷 낮아지지않음? 5차때처럼 5차스킬하...,152
3,2023-06-01 03:01,6차 나올때까지 노말 진듄다니기 vs 하드 진듄 2인팟 가기 어카지? 한달만 참어??,47
4,2023-06-01 03:34,6차 거의 확정으로보는 분위기 아녓음? 오늘은분위기가 갑자기 다들 회의적이네 냉정해졌어,48
...,...,...,...
15070,2023-06-30 23:19,오늘 지하철 보니까 버츄얼 유튜버 생일도 챙기고 그러던데 우리도 6차 잘나오면 지하...,60
15071,2023-06-30 23:26,●▅▇█▇▆▅▄▇ 아 6차 배달 잘 오고 있나?,26
15072,2023-06-30 23:47,진짜 메이플 빼고 옆동네는 다 난리났네… 로아는 안그래도 망해가는데 중국몽 같이해버...,141
15073,2023-06-30 23:57,진짜 심각) 미하일 주력기 문의 한번만 더 봐주세요 재업 추하지만 저번에 올렸던 글...,134


In [ ]:
# 라벨링과 점수 생성
target_df[['Label', 'Score']] = target_df['Reaction'].apply(lambda x: pd.Series(sentiment(x)))
target_df

,Date,Reaction,Length,Label,Score
0,2023-06-01 01:02,6차 아니면 로드맵 유기하면 안되는거 아니야 간담회까지 열어서 로드맵 보여주겠다고 ...,176,0,99.650
1,2023-06-01 02:08,6차 왔냐 66 66 6 6 ;;;,19,0,92.741
2,2023-06-01 02:59,근데 6차나오면 전직업 최종뎀 뻥튀기되서 보스컷 낮아지지않음? 5차때처럼 5차스킬하...,152,0,60.217
3,2023-06-01 03:01,6차 나올때까지 노말 진듄다니기 vs 하드 진듄 2인팟 가기 어카지? 한달만 참어??,47,1,80.003
4,2023-06-01 03:34,6차 거의 확정으로보는 분위기 아녓음? 오늘은분위기가 갑자기 다들 회의적이네 냉정해졌어,48,0,68.620
...,...,...,...,...,...
15070,2023-06-30 23:19,오늘 지하철 보니까 버츄얼 유튜버 생일도 챙기고 그러던데 우리도 6차 잘나오면 지하...,60,1,96.456
15071,2023-06-30 23:26,●▅▇█▇▆▅▄▇ 아 6차 배달 잘 오고 있나?,26,1,69.761
15072,2023-06-30 23:47,진짜 메이플 빼고 옆동네는 다 난리났네… 로아는 안그래도 망해가는데 중국몽 같이해버...,141,0,89.979
15073,2023-06-30 23:57,진짜 심각) 미하일 주력기 문의 한번만 더 봐주세요 재업 추하지만 저번에 올렸던 글...,134,1,63.413


In [ ]:
target_df.sort_values('Length', inplace=True)
target_df.head(20)

,Date,Reaction,Length,Label,Score
2004,2023-06-10 13:17,6차 6,4,1,83.061
6804,2023-06-17 16:00,6차 6차,5,1,91.014
8248,2023-06-22 17:52,칼리 6차,6,1,89.554
10979,2023-06-23 12:51,썬콜6차 와,6,0,76.475
668,2023-06-02 19:13,6차확률 .,6,1,75.342
8170,2023-06-22 17:37,듀블 6차,6,1,93.332
133,2023-06-01 17:13,6차 6차는,6,1,75.181
8063,2023-06-22 17:13,6차 ui 내,7,0,54.352
14112,2023-06-28 03:24,6차수정좀 네,7,0,77.636
8092,2023-06-22 17:23,엔버6차 ㅇㅇ,7,0,59.580


문장의 길이가 8이하인 문장은 제거

In [ ]:
target_df.drop(target_df[target_df['Length'] <= 8].index, inplace=True)
target_df

,Date,Reaction,Length,Label,Score
0,2023-06-01 01:02,6차 아니면 로드맵 유기하면 안되는거 아니야 간담회까지 열어서 로드맵 보여주겠다고 ...,176,0,99.650
1,2023-06-01 02:08,6차 왔냐 66 66 6 6 ;;;,19,0,92.741
2,2023-06-01 02:59,근데 6차나오면 전직업 최종뎀 뻥튀기되서 보스컷 낮아지지않음? 5차때처럼 5차스킬하...,152,0,60.217
3,2023-06-01 03:01,6차 나올때까지 노말 진듄다니기 vs 하드 진듄 2인팟 가기 어카지? 한달만 참어??,47,1,80.003
4,2023-06-01 03:34,6차 거의 확정으로보는 분위기 아녓음? 오늘은분위기가 갑자기 다들 회의적이네 냉정해졌어,48,0,68.620
...,...,...,...,...,...
15070,2023-06-30 23:19,오늘 지하철 보니까 버츄얼 유튜버 생일도 챙기고 그러던데 우리도 6차 잘나오면 지하...,60,1,96.456
15071,2023-06-30 23:26,●▅▇█▇▆▅▄▇ 아 6차 배달 잘 오고 있나?,26,1,69.761
15072,2023-06-30 23:47,진짜 메이플 빼고 옆동네는 다 난리났네… 로아는 안그래도 망해가는데 중국몽 같이해버...,141,0,89.979
15073,2023-06-30 23:57,진짜 심각) 미하일 주력기 문의 한번만 더 봐주세요 재업 추하지만 저번에 올렸던 글...,134,1,63.413


In [ ]:
a = target_df

## 1. 길이가 긴 문장은 어떻게 해야하는가?

In [ ]:
target_df[target_df['Length'] >= 1000]

,Date,Reaction,Length,Label,Score
131345,2023-06-23 17:56,문제가 보니까 극딜 압축 잘 됐던 직업 : 지속딜 6차극딜 지속이 긴 직업 : 한방...,1002,1,62.431
50998,2023-06-11 02:29,늦은 CGV 후기 처음으로 간 메이플 오프라인 이벤트라 기억해두고싶어서 쓰는 글1부...,1014,1,98.784
153824,2023-06-27 08:36,오리진 스킬의 문제점 1. 리레/프레이 의존성-> 현재 메이플 보스전투환경은 파티플...,1026,0,98.450
150937,2023-06-26 20:07,6차 전직 직업별 개선안 [마법사] 각 직게에서 추천을 가장 많이 받았거나 제대로 ...,1044,1,66.606
32210,2023-06-09 01:30,내일 쇼케 희망사항 1. 음악회를 열다2. 콘서트를 열다3. 대 성 캐 출현4. 6...,1060,1,66.076
...,...,...,...,...,...
140454,2023-06-25 04:11,복귀 뉴비의 20년 메생과 지금 상황 (장문) 밑에 요약 있음메이플스토리라는 게임은...,6937,1,86.259
152384,2023-06-26 23:54,●▅▇█▇▆▅▄▇ 엔버 살려조ㅠㅠ(개선안) 엔버 살려주세요...극딜 구조 너무 암울...,7033,1,51.329
144442,2023-06-25 20:32,70층 팬텀이 쓰는 팬텀 문제점 및 개선안 (머슴밥 주의) 잘못된 정보가 있으면 댓...,11560,0,76.069
77818,2023-06-14 21:58,명예훈장 X 강원기 합방 내용 정리 합방을 바쁘게 준비해주신 명예훈장님께 수고하셨다...,20346,1,90.676


결론 - 점수가 60점 미만인 문장을 제외하면 어느정도 정확도가 높다.   (패스)

## 2. 질문글은 어떻게 해야하는가?

In [ ]:
# '?'단어만 포함된 데이터 프레임 추출
question_df = target_df[target_df['Reaction'].str.contains('\?')]
question_df.sort_values('Date')

,Date,Reaction,Length,Label,Score
0,2023-06-01 01:02,6차 아니면 로드맵 유기하면 안되는거 아니야 간담회까지 열어서 로드맵 보여주겠다고 ...,176,0,99.714
2,2023-06-01 02:59,근데 6차나오면 전직업 최종뎀 뻥튀기되서 보스컷 낮아지지않음? 5차때처럼 5차스킬하...,152,0,68.956
3,2023-06-01 03:01,6차 나올때까지 노말 진듄다니기 vs 하드 진듄 2인팟 가기 어카지? 한달만 참어??,47,1,79.190
4,2023-06-01 03:34,6차 거의 확정으로보는 분위기 아녓음? 오늘은분위기가 갑자기 다들 회의적이네 냉정해졌어,48,0,77.222
10,2023-06-01 05:04,내가보긴 6차 안나올듯.. 대신 레드때처럼 6차 웜업? 영상 올라올거같음이새끼들 개...,90,0,81.517
...,...,...,...,...,...
15065,2023-06-30 22:29,6차극딜 퍼뎀이70만 100만이러던데 얼마나쌘거임? 나로 스프풀히트가 몇만되려나일반...,64,1,66.176
15067,2023-06-30 22:50,이번주 주말 점유율 최고점 아닐까?싶은 6차 나오기전까지 왠지 이번주 최고점 될수있...,66,1,93.590
15071,2023-06-30 23:26,●▅▇█▇▆▅▄▇ 아 6차 배달 잘 오고 있나?,26,1,63.206
15072,2023-06-30 23:47,진짜 메이플 빼고 옆동네는 다 난리났네… 로아는 안그래도 망해가는데 중국몽 같이해버...,141,0,90.445


질문글을 모두 지울 수는 없다. 이것도 60점 미만 제거.   (패스)

## 3. 투표글은 어떻게 해야하는가?

In [ ]:
# 'ㅌㅍ'단어만 포함된 데이터 프레임 추출
vote_df = target_df[target_df['Reaction'].str.contains('ㅌㅍ')]
vote_df.sort_values('Date')

,Date,Reaction,Length,Label,Score
14,2023-06-01 08:20,ㅌㅍ) 6차안나오면 피시방점유율 몇퍼 예상함? 리부트 피케인 안준다는 가정하에6차도...,51,0,87.295
43,2023-06-01 11:02,ㅌㅍ) 6차뜨기 VS 돼지국밥에 공기밥 말아 국물까지 뚝딱하기 뭐가 더 낫냐?,43,1,50.119
187,2023-06-01 18:14,ㅌㅍ)이번 쇼케때 6차 나올거같음?? ㄷㄱㄷㄱ,25,1,82.650
513,2023-06-02 12:31,ㅌㅍ) 6차 최종뎀 상승량은 몆퍼가 적당함? ㄱㄱ,27,1,60.945
1025,2023-06-06 20:05,ㅌㅍ) 쇼케 6차 뜬다 안뜬다 ㅇㅇ,19,0,60.008
4979,2023-06-12 00:27,6차쿨타임 예상.ㅌㅍ 드가자,15,1,94.935
7684,2023-06-22 00:39,"6차쿨.ㅌㅍ 뭔가 6차는 메르, 쿨감 적용 안될것 같은 느낌",33,0,86.026


In [ ]:
# '투표'단어만 포함된 데이터 프레임 추출
vote2_df = target_df[target_df['Reaction'].str.contains('투표')]
vote2_df.sort_values('Date')

,Date,Reaction,Length,Label,Score
125,2023-06-01 17:01,이번 쇼케이스에 6차 나올까 투표해줘,20,1,91.660
151,2023-06-01 17:40,6차 나올 것 같음? 투표해줘,16,1,91.207
244,2023-06-01 18:58,6차 예상 레벨 투표 스토리상275인데 진입장벽상 260예상,33,0,81.035
732,2023-06-03 22:19,쇼케이스 큰거 순위 투표 ㅈ박은 성장경험 개선신규 장기 콘텐츠 출시주간 숙제로 전락...,844,1,77.174
749,2023-06-04 11:39,농장투표 이니이벤트 조합용 농장 하나 더 만들건데투표하고 주사위굴려서 6 나오면 선...,129,1,51.247
...,...,...,...,...,...
13661,2023-06-27 01:05,6차패치 선호도 조사 결과 6차패치 평점 설문 결과입니다!결과 구하는 과정에서 확인...,227,1,89.872
13963,2023-06-27 14:06,6차 스킬 컷신 일러스트 관련 투표해봐요! 개인적으로는 스킬 컷신에 직업 일러스트 ...,131,1,95.337
14137,2023-06-28 09:33,전사 6차 투표하자 어떤게 가장 멋있냐?,22,1,60.865
14446,2023-06-28 19:51,6차공용쿨 투표 ㄱㄱ,11,1,96.484


In [ ]:
# 'vs'단어만 포함된 데이터 프레임 추출
vote3_df = target_df[target_df['Reaction'].str.contains('vs')]
vote3_df.sort_values('Date')

,Date,Reaction,Length,Label,Score
3,2023-06-01 03:01,6차 나올때까지 노말 진듄다니기 vs 하드 진듄 2인팟 가기 어카지? 한달만 참어??,47,1,80.003
73,2023-06-01 13:42,6차 나온다 vs 안나온다 ㄱㄱ,17,0,88.641
82,2023-06-01 15:18,"여름방학 자전코인.. 나온다vs안나온다 이번 여름방학, 6차전직이 기정사실화 되어있...",138,0,85.365
128,2023-06-01 17:39,6차 vs 신직업 .,11,0,65.733
218,2023-06-01 19:09,쇼케이스 6차 나오기 vs 안나와서 화난 메벤 보기 머가더 재밌음??,38,0,96.408
...,...,...,...,...,...
12950,2023-06-28 22:02,하버 엔버 vs 캐슈 엔버 6차랑 컨셉도 끌리고 캐슈 타격감이랑 다 끌리는데진짜 너...,62,0,67.960
13045,2023-06-29 12:09,일주일만에 싹 뜯어고치고 테섭 나오는건 어려운거 다 알아 그러니깐 1. 직업별로 어...,182,0,98.776
13055,2023-06-29 13:43,님들이 생각하는 6차스킬 공용쿨타임 개선방안은 뭐임 지금 쿨타임 그대로 들여오고 6...,99,0,92.378
13250,2023-06-29 21:35,리vs본 돌아온 찍먹연어 둘다 생성가능본썹은 유뇬3000정도 리는 1000정도...,164,1,92.669


투표글 삭제

vs는 문장 길이가 100이하인 문장 제거

## 4. 정보글은 어떻게 해야하는가?

In [ ]:
# '정보'단어만 포함된 데이터 프레임 추출
info1_df = target_df[target_df['Reaction'].str.contains('정보')]
info1_df.sort_values('Date')

,Date,Reaction,Length,Label,Score
434,2023-06-02 10:38,아 시발 이새끼들 콘서트 불태우는 이유 알았다 걍 댓글 ㅈㄴ 읽어보니까 대충 파악 ...,252,0,86.129
1318,2023-06-07 23:32,정보 유트버 마저 6차다 6차안하면 수만명이 죽음,27,0,81.454
3100,2023-06-10 19:00,6차관해서 지금 아무정보도 없다고!!! 쇼케에 헥사매트릭스 설명좀 나온거빼면 스킬에...,80,0,96.554
3570,2023-06-10 23:21,아직 6차 정보 없지? 스킬 퍼뎀이라던가 얼만큼 세진다던가 그런거,36,0,84.136
3951,2023-06-11 01:55,하이퍼버닝 6차스킬정보 모른 채로 시작하나요? 하버 줘도 기다리고 있다가 테섭 보고...,55,0,84.885
4779,2023-06-11 19:03,이번 쇼케이스 주관적 평가 본인소개부터 먼저본섭/리부트 둘다 276+/유니온 8천 ...,2617,1,68.806
4793,2023-06-11 19:20,지금 강원기한테 해서 들을수 있는 정보는 이게 최선아님? 6차 연출 그거 성별마다 ...,149,0,51.794
4804,2023-06-11 19:39,6차 테섭은 바로 풀어줬어도 됐을텐데 ㄹㅇ 하버캐릭 정하는데 6차 오리진스킬 연출만...,152,0,95.839
5082,2023-06-12 02:59,다음주가 하버고 한달뒤가 6차인데 6차 정보 모르고 하버 키우는거임?? 6차 잘나온...,80,0,95.900
5404,2023-06-12 15:14,리퐁보다 재밌는 6차예측(뇌피셜가득 의견환영) 옛날엔 이런거 나오자마자 분석해주시는...,1933,1,95.223


패스

In [ ]:
# '요약'단어만 포함된 데이터 프레임 추출
info2_df = target_df[target_df['Reaction'].str.contains('요약')]
info2_df.sort_values('Date')

,Date,Reaction,Length,Label,Score
137,2023-06-01 17:21,"쇼케 진행 예상 1. 성캐, 원기/창섭 등장, 대충 2분간 주절거림- 메이플이 용사...",1126,1,61.123
639,2023-06-02 17:31,흰딱이가 6월에 6차 올거라고 생각하는 eu. 뭐 스토리라인이나 포스터 떡밥은 많은...,804,1,63.071
811,2023-06-05 09:31,"리퐁부터 지금까지 사건을 토대로 쇼케 요약하면 이거임? 6월 쇼케에 6차, 전투력지...",95,0,68.971
888,2023-06-05 15:56,리부트 너프하라는이유와 6차 안나오는이유 본섭유저 입장에서 리부트 너프하라는말이 나...,1383,0,97.457
937,2023-06-05 21:30,이번 쇼케이스 요약 리부트너프x6차x메소값 떡락템가격 떡락,32,0,85.724
...,...,...,...,...,...
14788,2023-06-29 17:49,[필독] 라이브 간단 요약..txt 1. 6차 60초쿨 삭제2. 바인드 완화 (기존...,318,1,85.567
14793,2023-06-29 17:53,"라이브 초간단 요약 ★6차1. 공용쿨 삭제2. 절대 바인드, 일반 바인드 덮을 수 ...",631,1,74.380
14796,2023-06-29 17:56,오늘 라이브 요약 이거 맞음? ---- 6차관련 ----1. 6차 파티전체 60초 ...,663,1,73.338
14804,2023-06-29 18:01,라이브 요약 서론1. 이번주 테섭 없다.2. 미안하다. 미흡한 점이 많았다.수치는 ...,3159,1,68.661


패스

In [ ]:
# '요약'단어만 포함된 데이터 프레임 추출
info3_df = target_df[target_df['Reaction'].str.contains('정리')]
info3_df.sort_values('Date')

,Date,Reaction,Length,Label,Score
156,2023-06-01 17:46,6차 안나오면 메소 떡락해? 천억 들고있는데 지금 정리할까,32,1,68.727
542,2023-06-02 13:58,토끼 아니마 궁수 / 6차전직 / 깜짝 리마스터 공개 현재 떡밥 정리 해둔 사람 ...,58,1,75.921
588,2023-06-02 15:41,6차 기대하기보다 경기장 출입구봉쇄 유저와 디렉터간 6000:2 서열정리 케이지매치...,78,0,90.215
601,2023-06-02 16:01,뇌피셜) 본섭이랑 리부트 서버간의 격차는 6차 전직으로 해결 할듯 우선 6차가 이번...,412,1,53.902
606,2023-06-02 16:08,사실 신규 스펙업 수단이 나온다는 건 확실하지 다들 비슷한 생각 하고 있겠지만 그래...,401,0,89.162
...,...,...,...,...,...
13862,2023-06-27 12:16,이번 6차 딜구조 정리해드림 단타>>버프>>>>>>설치기설치기 받은 직업들은 진짜 ...,59,0,79.946
14559,2023-06-28 23:12,"6차 나오면 유저들이 해야할일 정리 게임 욕 O개발자 욕 O원기 , 창섭 욕 O본인...",96,1,54.751
14785,2023-06-29 17:47,6/29 메이플라이브 정리사항 6/29 메이플라이브 정리사항1.이제 6차 공용쿨 삭...,538,1,91.988
14787,2023-06-29 17:48,라이브 내용 정리 1. 6차 스킬 변경사항 오류인지 아닌지 유저에게 명확하게 안알...,1728,1,65.189


패스

In [ ]:
# '요약'단어만 포함된 데이터 프레임 추출
info4_df = target_df[target_df['Reaction'].str.contains('스압')]
info4_df.sort_values('Date')

,Date,Reaction,Length,Label,Score
1374,2023-06-08 10:56,6차행복회로(스압) 아이디어 가져온곳메이플 늬온-신의창 프로토콜https://you...,143,1,96.324
11683,2023-06-23 21:51,약스압)배틀메이지 6차 후기 및 문제점.review 아니 배메는 잘 받은거 아니었음...,2298,1,74.626
15074,2023-06-30 23:58,"(스압,데이터주의) 멮지지 배경 15종(듀블,불썬숍,아란,제논,카데나,캡틴,키네,팔...",269,1,84.633


패스

## 5. 필요없는 단어

In [ ]:
# '주사위'단어만 포함된 데이터 프레임 추출
dice_df = target_df[target_df['Reaction'].str.contains('주사위')]
dice_df.sort_values('Date')

,Date,Reaction,Length,Label,Score
18,2023-06-01 10:05,6차 예고? 주사위 눈 : 6,16,1,95.625
374,2023-06-01 22:11,이번 쇼케 6차 나올 확률 오랜만에 주사위,23,1,97.451
454,2023-06-02 10:56,메붕이들 6차 추측하는걸 보니까 쇼케짤이든 인벤에서 주사위든 게임 안에서든 6이라는...,121,1,66.918
528,2023-06-02 13:18,이번 여름방학은 6차+영웅리마임 뭐 그렇게 6차근들갑 떠는거 알고도 키네시스 주사위...,186,0,95.115
532,2023-06-02 13:35,6차 나올 확률 /주사위,13,1,83.318
749,2023-06-04 11:39,농장투표 이니이벤트 조합용 농장 하나 더 만들건데투표하고 주사위굴려서 6 나오면 선...,129,1,51.247
790,2023-06-04 21:47,메이플 그림채 ai 대충해드림 주사위6 연습용으로 3명만 해드림 단 얼굴만 한정해서...,171,0,64.018
955,2023-06-06 02:12,주사위 66뜨는사람 이니 드림 선착순 단한명 만이니 가져가온세상이 6차다,40,1,74.461
976,2023-06-06 11:10,주사위가 홀수면 6차전직 없다 짝수면 나온다는 말.,28,1,84.598
1421,2023-06-08 15:27,인벤탈퇴빵 주사위 ㄱㄱ 6차기념 66나오면 탈퇴 ㄱㄱ,29,0,57.877


주사위는 메이플 인벤에서 사용하는 말그대로 주사위 시스템 -> 해당 행 삭제

더 많겠지만 여기까지 정리 완료. 투표, ㅌㅍ와 주사위행만 제거

In [ ]:
target_df = target_df[~target_df['Reaction'].str.contains('ㅌㅍ')]
target_df = target_df[~target_df['Reaction'].str.contains('투표')]
target_df = target_df[~target_df['Reaction'].str.contains('주사위')]
target_df = target_df.drop(target_df[(target_df['Reaction'].str.contains('vs')) & (target_df['Length'] < 100)].index)
target_df['Score'] = target_df['Score'].astype(float)
target_df = target_df[target_df['Score'] >= 60]
target_df = target_df.reset_index(drop=True)
target_df

,Date,Reaction,Length,Label,Score
0,2023-06-01 01:02,6차 아니면 로드맵 유기하면 안되는거 아니야 간담회까지 열어서 로드맵 보여주겠다고 ...,176,0,99.650
1,2023-06-01 02:08,6차 왔냐 66 66 6 6 ;;;,19,0,92.741
2,2023-06-01 02:59,근데 6차나오면 전직업 최종뎀 뻥튀기되서 보스컷 낮아지지않음? 5차때처럼 5차스킬하...,152,0,60.217
3,2023-06-01 03:34,6차 거의 확정으로보는 분위기 아녓음? 오늘은분위기가 갑자기 다들 회의적이네 냉정해졌어,48,0,68.620
4,2023-06-01 04:16,내 닉네임 검색하다가 레전드 글발견함ㅋㅋ https://m.inven.co.kr/b...,127,1,83.405
...,...,...,...,...,...
13326,2023-06-30 23:19,오늘 지하철 보니까 버츄얼 유튜버 생일도 챙기고 그러던데 우리도 6차 잘나오면 지하...,60,1,96.456
13327,2023-06-30 23:26,●▅▇█▇▆▅▄▇ 아 6차 배달 잘 오고 있나?,26,1,69.761
13328,2023-06-30 23:47,진짜 메이플 빼고 옆동네는 다 난리났네… 로아는 안그래도 망해가는데 중국몽 같이해버...,141,0,89.979
13329,2023-06-30 23:57,진짜 심각) 미하일 주력기 문의 한번만 더 봐주세요 재업 추하지만 저번에 올렸던 글...,134,1,63.413


In [ ]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13331 entries, 0 to 13330
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      13331 non-null  object 
 1   Reaction  13331 non-null  object 
 2   Length    13331 non-null  int64  
 3   Label     13331 non-null  object 
 4   Score     13331 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 520.9+ KB


# 긍정 부정 레이블이 유의한지 확인

In [ ]:
target_df

,Date,Reaction,Length,Label,Score
0,2023-06-01 01:02,6차 아니면 로드맵 유기하면 안되는거 아니야 간담회까지 열어서 로드맵 보여주겠다고 ...,176,0,99.650
1,2023-06-01 02:08,6차 왔냐 66 66 6 6 ;;;,19,0,92.741
2,2023-06-01 02:59,근데 6차나오면 전직업 최종뎀 뻥튀기되서 보스컷 낮아지지않음? 5차때처럼 5차스킬하...,152,0,60.217
3,2023-06-01 03:34,6차 거의 확정으로보는 분위기 아녓음? 오늘은분위기가 갑자기 다들 회의적이네 냉정해졌어,48,0,68.620
4,2023-06-01 04:16,내 닉네임 검색하다가 레전드 글발견함ㅋㅋ https://m.inven.co.kr/b...,127,1,83.405
...,...,...,...,...,...
13326,2023-06-30 23:19,오늘 지하철 보니까 버츄얼 유튜버 생일도 챙기고 그러던데 우리도 6차 잘나오면 지하...,60,1,96.456
13327,2023-06-30 23:26,●▅▇█▇▆▅▄▇ 아 6차 배달 잘 오고 있나?,26,1,69.761
13328,2023-06-30 23:47,진짜 메이플 빼고 옆동네는 다 난리났네… 로아는 안그래도 망해가는데 중국몽 같이해버...,141,0,89.979
13329,2023-06-30 23:57,진짜 심각) 미하일 주력기 문의 한번만 더 봐주세요 재업 추하지만 저번에 올렸던 글...,134,1,63.413


중심 극한의 정리에 의해 30개 행 추출,

위 추출을 10번 반복

유의 수준은 KoBERT2에서 검증 데이터 정확도인 0.85

In [ ]:
import random

# 0에서 13330까지의 숫자 중 300개를 랜덤하게 선택
random_numbers = random.sample(range(0, 13331), 300)

# 30행 10열의 리스트로 재구성
num_rows = 10
num_cols = 30
result_list = [random_numbers[i*num_cols: (i+1)*num_cols] for i in range(num_rows)]

# 각 row를 다른 변수명으로 설정하여 출력
for i, row in enumerate(result_list):
    variable_name = f"num_{i+1}"
    exec(f"{variable_name} = row")
    print(f"{variable_name}: {row}")

num_1: [8313, 11208, 3632, 2446, 12411, 1801, 12030, 707, 11968, 13265, 1323, 7102, 6800, 13282, 7190, 5368, 3031, 10676, 11679, 11037, 11987, 2731, 4306, 11207, 6051, 2806, 9208, 5586, 6910, 11365]
num_2: [9520, 1257, 10818, 2194, 7580, 1526, 7001, 9859, 1427, 298, 4683, 11058, 6839, 2766, 4504, 1381, 9752, 12367, 4011, 6216, 13274, 3001, 1729, 6767, 8258, 8155, 283, 1518, 3213, 1072]
num_3: [3918, 1954, 9876, 1220, 323, 4126, 7038, 5478, 5800, 7096, 858, 968, 9646, 12132, 4821, 11303, 6649, 13323, 12772, 5395, 3634, 5436, 10646, 10010, 2633, 7505, 4097, 11398, 3808, 2493]
num_4: [5278, 8436, 6854, 1707, 12353, 5761, 11228, 11086, 5798, 9565, 8251, 10586, 2314, 5791, 7614, 3963, 8845, 11396, 4644, 6485, 6626, 5124, 511, 10692, 7161, 3165, 7635, 12158, 7556, 6506]
num_5: [5579, 12759, 1625, 1025, 1308, 7624, 10928, 11300, 9467, 10829, 9999, 3210, 6794, 13064, 6719, 13121, 704, 6898, 12050, 6301, 1282, 10882, 9328, 8221, 4054, 9458, 10423, 6718, 8632, 7243]
num_6: [9330, 7384, 4962, 107

## 첫번째

In [ ]:
target_df.loc[num_1]

,Date,Reaction,Length,Label,Score
8313,2023-06-22 21:35,아니 왜 이렇게 분탕질임? 6차 넘어오면 고쳐주겠지 믿고 기다리면 잘 할텐데하면 잘...,84,0,98.031
11208,2023-06-25 14:23,이 기세면 이번 주 목요일애 6차 추가패치 있겠지? 기대할게!!,35,1,99.452
3632,2023-06-11 04:34,구맥뎀 뚫었다고 신났는데 구구맥뎀됨 나도 패치 나올때마다 패치체감 해보고싶어6차 1...,55,0,78.213
2446,2023-06-10 18:04,후 6차는 5차랑 좀 다르게 내던가하지 뭔 또 매트리스...,33,0,85.948
12411,2023-06-27 19:26,애초에 극딜 밸런스를 맞출거였으면 ㅋㅋㅋ 컨티뉴를 그렇게 애매하게 내고 6차를 각성...,110,0,97.621
1801,2023-06-10 15:36,사람들 이번 쇼케에 6차 배율째고있는거 개무섭네 ㅋㅋㅋ 없으면 계란 날라올 듯 ㅋㅋ...,50,1,97.260
12030,2023-06-27 00:20,일리움 이 움짤 자세히는 첨 봤는데 개웃기네 어캐하면 6차 쓰다가 데카 2개가 까임...,55,1,81.697
707,2023-06-05 09:36,무조건 6차라고 6차임 그래야 낭만이니까,22,1,98.018
11968,2023-06-26 23:37,♡●▅▇█▇▅▄▄ 엔버살려줘 진짜로 스택쌓는데 10초간 평딜 치고 있다고그리고 극딜...,113,0,95.788
13265,2023-06-30 14:07,시드링 극딜주기에 따른 차이 해결법 6차 쿨이 6분이겠다 시드링도 6분으로 맞추면 될듯,48,1,97.966


27/30, 정확도 : 90%

## 두번째

In [ ]:
target_df.loc[num_2]

,Date,Reaction,Length,Label,Score
9520,2023-06-23 11:36,움직임을 봉인하는 바인를 무시하는 보스를 무시하는 6차바인드 를 무시하는 신규보스 ...,123,0,65.795
1257,2023-06-08 17:44,아까 10추글에서 본건데 5시 58분쯤에 성캐가 6시니까 6차를 서프라이즈로 공개한...,95,1,74.096
10818,2023-06-24 17:59,ㅋㅋㅋㅋ 로아 까려고 태세전환은 좀 그렇네 당장 몇시간 전만 해도 첫 테섭부터 불질...,134,0,86.816
2194,2023-06-10 17:30,6차 임팩트 누가 제일 쩔었냐 난 소마,21,1,77.838
7580,2023-06-22 19:06,6차 몰아서 못쓰는거면 한명이 6차쓰면 이펙트 공유됨? 파티원한테 다보임?,41,1,61.500
1526,2023-06-09 17:45,"6차, 리메이크, 신직업 영웅, 레지, 카이저 엔버 리메이크 1차토끼 아니마 신직업...",97,1,96.439
7001,2023-06-22 16:38,오늘 6차 테섭나오고 인벤 ㅈㄴ 불날거 기대되는 메붕이면 개추 일단 데몬 집부터ㅋㅋ,46,1,95.872
9859,2023-06-23 15:08,???: 6차스킬 써도 하나도 바뀐게 없는데요?? 진짠뎅,31,0,96.904
1427,2023-06-09 11:57,6차전직 또는 리마스터해도 가불기인이유 2가지 어차피 어센틱심볼 안옮겨주면 후후후후...,86,0,98.452
298,2023-06-01 21:15,리마스터 끝난 모험가랑 시그만 6차나오면 불탐?,26,0,68.606


28/30, 정확도 : 93%

## 세번째

In [ ]:
target_df.loc[num_3]

,Date,Reaction,Length,Label,Score
3918,2023-06-11 13:21,근데 진짜 빨간약 사태가 대단하긴 했나보다 6차에 경험치 개선 기타등등 이정도면 대...,199,0,99.546
1954,2023-06-10 16:21,6차전직 팩트 어차피 멋있어도 맨날 재획해고 똑같은 보스 잡는건 변함없음ㅋ6차스킬도...,65,0,90.413
9876,2023-06-23 15:22,보마야 고마워 5차까지 게이지 직업이 한둘이 아닌데6차에 게이지가 아예 없는건 말이...,92,1,96.132
1220,2023-06-08 13:06,6차아님? 나올거가있음???,15,0,74.462
323,2023-06-01 22:18,6차 안 나오면 우리가 할 수 있는 것.txt 1. 그동안 뭐했냐고 메벤에 찡찡거리...,229,1,88.358
4126,2023-06-11 17:44,"버닝섭에서 하이퍼 키울건데 마음에 둔것중 추천좀 나로, 은월, 바이퍼, 캡틴, 윈브...",120,1,94.792
7038,2023-06-22 17:12,6차 업적은 안주나보네 업적 변동사항 없음,23,0,76.253
5478,2023-06-14 19:13,에반 6차 테섭기대하라는데 설마..? 이런식으로 조디악나오냐? 기대중,38,1,94.932
5800,2023-06-16 13:46,아직 6차 안나왔는데 점유율 뭔데 개쩐다,22,1,98.531
7096,2023-06-22 17:32,2분 극딜 불합리한점 2분마다 극딜을 쓰면첫극 리레 6차극딜두번째 5차극딜 웨펖세번...,134,0,89.799


28/30, 정확도 : 93%

## 네번째

세개만 돌려봐도 (0.9 + 0.93 + 0.93) / 3 = 0.92로 표본의 정확도가 좋아서 모집단의 정확도도 좋다고 판단.

In [ ]:
# 저장
target_df.to_csv('preprocessed_maple.csv', index=False)